In [ ]:
import os
import numpy as np
import mne
from mne.io import read_raw_cnt
import soundfile as sf

In [ ]:
# Set directories
field_trip_directory = r'C:\Users\Mahmud\Documents\fieldtrip-20220729'
my_directory = r'E:\Research\EEG_Data\ano'

# Get Participant ID
participant_id = input("Input participant ID number: ")

# Set file path and data list
data_list = [f for f in os.listdir(my_directory) if f.startswith('2029') and 'ano' in f and f.endswith('.cnt')]

# Initialize a list to store the processed data
processed_data_list = []

# Loop through each file in the data list and preprocess it
for file in data_list:
    file_path = os.path.join(my_directory, file)
    print(f"Processing file: {file_path}")
    
    # Read the CNT file
    raw = read_raw_cnt(file_path, preload=True)

    # Drop unwanted channels
    drop_channels = ['M1', 'M2', 'EMG', 'REF', 'COMNT', 'SCALE']
    raw.drop_channels([ch for ch in drop_channels if ch in raw.ch_names])

    # Set the reference
    ref_channels = [ch for ch in raw.ch_names if ch not in ['EKG', 'EMG', 'HEO', 'VEO']]
    raw.set_eeg_reference(ref_channels, projection=False)

    # Apply notch filter at 60 Hz
    raw.notch_filter(freqs=60)

    # High-pass filter
    raw.filter(l_freq=0.1, h_freq=None, fir_design='butterworth', filter_length='auto')

    # Low-pass filter
    raw.filter(l_freq=None, h_freq=100, fir_design='butterworth', filter_length='auto')

    # Define events
    events = mne.find_events(raw, stim_channel='STI 014', output='onset', min_duration=0.002)
    
    # Define epochs based on events
    event_id = {str(i): i for i in range(121)}  # Trial markers
    tmin, tmax = -3, 4  # Pre-stimulus and post-stimulus
    epochs = mne.Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True)

    # Resample the data to 500 Hz
    epochs_resampled = epochs.copy().resample(500, npad="auto")
    
    processed_data_list.append(epochs_resampled)

In [ ]:
# Concatenate all epochs
all_epochs = mne.concatenate_epochs(processed_data_list)

# Save the preprocessed data
save_directory = input("Enter the directory where you want to save the preprocessed data: ")
save_name = f'data_final_iso_{participant_id}.fif'
all_epochs.save(os.path.join(save_directory, save_name), overwrite=True)

# Count how many trials there are for a specific condition (e.g., condition 21)
condition_count = np.sum(all_epochs.events[:, 2] == 21)
print(f"Number of trials for condition 21: {condition_count}")